In [36]:
import random
import speech_recognition as sr
from gtts import gTTS
import Levenshtein
import numpy as np
from tabulate import tabulate

def generate_paragraph():
    sentences = [
        "The sun was shining brightly in the clear blue sky.",
        "A gentle breeze rustled the leaves of the tall trees.",
        "Children were playing in the park, laughing and enjoying their time.",
        "The smell of fresh flowers filled the air, creating a pleasant atmosphere.",
        "People strolled along the peaceful path, taking in the beauty of nature.",
        "A small cafe nearby offered delicious pastries and aromatic coffee.",
        "Birds chirped happily, adding to the symphony of nature's sounds.",
        "As the day progressed, the sky painted itself with warm hues of orange and pink.",
        "It was a serene evening, perfect for reflection and relaxation.",
    ]
    selected_sentences = random.sample(sentences, 2)
    paragraph = ' '.join(selected_sentences)
    return paragraph


def recognize_speech():
    recognizer = sr.Recognizer()
    recognized_texts = []  # Store recognized texts in a list

    with sr.Microphone() as source:
        print("\nPlease repeat the above paragraph:")
        audio = recognizer.listen(source, timeout=10, phrase_time_limit=10)  # Adjust these values as needed
        audio_copy = audio

    try:
        user_text = recognizer.recognize_google(audio)
        recognized_texts.append(user_text)  # Append recognized text to the list
        return user_text
    except sr.UnknownValueError:
        print("Sorry, could not understand audio.")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

def evaluate_pronunciation(generated_text, user_text):
    distance = Levenshtein.distance(generated_text, user_text)
    max_length = max(len(generated_text), len(user_text))
    similarity_ratio = 1 - (distance / max_length)
    return similarity_ratio

def calculate_single_fluency_score(recognized_text, target_text):
    distance = Levenshtein.distance(recognized_text, target_text)
    max_length = max(len(recognized_text), len(target_text))
    fluency_score = 1 - (distance / max_length)
    return fluency_score


if __name__ == "__main__":
    recognized_texts = []  # Initialize the list to store recognized texts

    generated_paragraph = generate_paragraph()
    print("Generated paragraph:")
    print(generated_paragraph)

    user_response = recognize_speech()
    if user_response:
        print("\nYour response:")
        print(user_response)

        similarity_ratio = evaluate_pronunciation(generated_paragraph, user_response)
        print(f"\nPronunciation Similarity Ratio: {similarity_ratio:.2%}")

        recognized_texts.append(user_response)  # Append recognized text to the list

        # Calculate the fluency score
        single_fluency_score = calculate_single_fluency_score(user_response, generated_paragraph)

        # Display fluency score and interpretation in a table
        table = [
            ["Fluency Score", single_fluency_score],
            ["Fluency Interpretation", 
                "Excellent fluency" if 0.90 <= single_fluency_score <= 1.00
                else "Good fluency" if 0.80 <= single_fluency_score < 0.90
                else "Moderate fluency" if 0.70 <= single_fluency_score < 0.80
                else "Fair fluency" if 0.60 <= single_fluency_score < 0.70
                else "Poor fluency" if 0.50 <= single_fluency_score < 0.60
                else "Very poor fluency, potential understanding issues"
            ]
        ]

        # Print the table
        print(tabulate(table, headers="firstrow", tablefmt="grid"))

    else:
        print("\nSpeech recognition failed.")


Generated paragraph:
A gentle breeze rustled the leaves of the tall trees. Birds chirped happily, adding to the symphony of nature's sounds.

Please repeat the above paragraph:

Your response:
a gentle Breeze result the leaves of the tall trees Birds chopped happily adding to the symphony of nature's sound

Pronunciation Similarity Ratio: 89.92%
+------------------------+----------------------+
| Fluency Score          | 0.8991596638655462   |
+========================+======================+
| Fluency Interpretation | Good fluency         |
+------------------------+----------------------+
